In [1]:
import pandas as pd
from datasets import Dataset, load_from_disk
import torchvision.transforms as transforms
import json
from PIL import Image
import os
import re

c:\Users\achia\anaconda3\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("data/coco/annotations/captions_train2017.json",'r') as fh:
    raw_train_data = json.load(fh)
with open("data/coco/annotations/captions_val2017.json",'r') as fh:
    raw_val_data = json.load(fh)

In [3]:
train_df = pd.DataFrame(raw_train_data["annotations"])
train_df.drop_duplicates(subset=['image_id'], inplace=True)
# train_df = train_df[:25000]
val_df = pd.DataFrame(raw_val_data["annotations"])
val_df.drop_duplicates(subset=['image_id'], inplace=True)

In [4]:
def map_image_id_to_image(image_id, split='train'):
    image_folder = f'data/coco/{split}2017/'
    # format image id with leading zeros
    filename = ""
    for i in range(12-len(str(image_id))):
        filename += '0'
    filename += str(image_id)
    formatted_image_id = re.sub(r'^(\d+)$', lambda x: x.group(1).zfill(6), filename)
    image_filename = f"{formatted_image_id}.JPG"  # Assuming the file extension is '.jpeg'
    return os.path.join(image_folder, image_filename)

# Apply the function to create the new column
train_df['img_path'] = train_df['image_id'].apply(map_image_id_to_image)
train_df['is_file'] = train_df['img_path'].apply(lambda x : os.path.isfile(x))
train_df = train_df[train_df['is_file'] == True]
val_df['img_path'] = val_df['image_id'].apply(lambda x: map_image_id_to_image(x, split='val'))
val_df['is_file'] = val_df['img_path'].apply(lambda x : os.path.isfile(x))
val_df = val_df[val_df['is_file'] == True]

In [5]:
# transform = transforms.Compose(
#         [transforms.ToTensor(),
#         transforms.Resize((128, 128)),
#         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

def preprocess_img(data_entry):
    raw  = Image.open(data_entry['img_path']).convert("RGB")
#     processed = transform(raw)
#     return {'raw_image': raw, 'img': processed}
    return {'raw_image': raw}

In [6]:
train_dataset = Dataset.from_pandas(train_df, preserve_index=False)
val_dataset = Dataset.from_pandas(val_df, preserve_index=False)

train_dataset = train_dataset.map(preprocess_img)
train_dataset.save_to_disk("processed_train")

Saving the dataset (115/115 shards): 100%|██████████| 118287/118287 [04:35<00:00, 429.37 examples/s]


In [7]:
val_dataset = val_dataset.map(preprocess_img)
val_dataset.save_to_disk("processed_val")

Saving the dataset (5/5 shards): 100%|██████████| 5000/5000 [00:04<00:00, 1202.31 examples/s]
